In [4]:
%matplotlib inline
import sys
sys.path.append('../shared')
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import copy
import model_normal_simple as mns
import model_vae_bayes as mvb
import elbo_calculator as ec
import train
import argparse
import os
from scipy.io import loadmat
import bayes_vae_tests as tests
%autoreload 2
test_case_name='frey1'
%load_ext autoreload

UsageError: Line magic function `%autoreload` not found.


In [5]:
%load_ext autoreload
test_case_name='frey1'
encoder = keras.models.load_model('%s_encoder_pretrained.h5' % test_case_name)
decoder = keras.models.load_model('%s_decoder_pretrained.h5' % test_case_name)

In [6]:
from ipywidgets import FloatSlider, interact
humour = FloatSlider(min=-15, max=15, step=3, value=0)
pose = FloatSlider(min=-15, max=15, step=3, value=0)
@interact(pose=pose, humour=humour)
def do_thumb(humour, pose):
    z = np.array([[pose, humour]]) * 0.05
    mu_x, log_sigma_x = decoder(z)
    face = np.reshape(mu_x.numpy(), (28,20))
    plt.figure(figsize=(2, 2))
    ax = plt.subplot(111)
    ax.imshow(face)
    plt.axis("off")

interactive(children=(FloatSlider(value=0.0, description='humour', max=15.0, min=-15.0, step=3.0), FloatSlider…

In [ ]:
%load_ext autoreload
def train_frey1():
    test_case_name='frey1'
    test_case = tests.test_cases[test_case_name]
    epochs = test_case['epochs']
    batch_size = test_case['batch_size']
    input_shape = test_case['input_shape']
    x = test_case['dataset_loader'](input_shape, test_case.get('options'))
    if test_case['model_class'] == mns.model_normal_simple:
        model = mns.model_normal_simple()
    else:
        dim_z_hidden, dim_z, dim_x_hidden = test_case['model_dims']
        model = test_case['model_class'](
            dim_z_hidden=dim_z_hidden,
            dim_z=dim_z,
            input_shape=x.shape[1:],
            dim_x_hidden=dim_x_hidden)
    optimizer = tf.keras.optimizers.Adamax()
    options = test_case.get('options')
    report_frequency=test_case.get('report_frequency',1)
    for epoch in range(epochs):
        loss_func = ec.elbo_calculator().get_loss_func()
        loss = train.train_one_random_batch(model, x, optimizer, loss_func, y=None, batch_size=batch_size, options=options)
        if ((epoch + 1) % report_frequency == 0):
            print('epoch %s: loss = %s' %(epoch+1, -loss.numpy()))
            if ((epoch + 1) % (report_frequency*10) == 0):
                model.encoder.save('%s_encoder.h5' % test_case_name)
                model.decoder.save('%s_decoder.h5' % test_case_name)
                z = np.random.normal(0,1,(1,dim_z))
                mu_x, log_sigma_x = model.decoder(z)
                img = np.reshape(mu_x.numpy(), (28,20))
                plt.imshow(img)
                plt.show()